# Code created to cleanse the Affected Pop Data

The data for this script comes daily, and the incountry IM are providing it through signal. The main folder are:
1. Orginal data: *D:\\NextCloud\\Projects\\2023\\2023-per-001\\GIS\\1_Original_Data\\115_UNDAC\\Pop_Affect*
2. Final output: *D:\\NextCloud\\Projects\\2023\\2023-per-001\\GIS\\2_Active_Data\\203_affd*

<code style="background:yellow;color:black">This notebook only works for Lambayeque, Piura and Tumbe. To make it work for the whole country a couple more edits in the source data need to take place. Please speak to Luis if this needs doing</code>

<code style="background:yellow;color:black">Please make sure to complete the variables below before running the notebook.  The date for the data is as per the file name sent by UNDAC</code>

In [21]:
DATA_DATE = '2023_04_20'
FILE_LOCATION = '20230420\\RS Nro 125_NACIONAL - REPORTE DE DANOS LLUVIAS 2023_20ABR2023_MA.xlsx'


## 1. Import packages

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path


In [22]:
## Setting Paths and Files
fldr_path = Path('D:\\NextCloud\\Projects\\2023\\2023-per-001\\GIS\\1_Original_Data\\115_UNDAC\\Pop_Affect')
fldr_outp = Path('D:\\NextCloud\\Projects\\2023\\2023-per-001\\GIS\\2_Active_Data\\203_affd')

file_path = Path(fldr_path / f'{FILE_LOCATION}')
file_pcod = Path(fldr_outp / 'per_pcode_reference_es.xlsx')
file_name = Path(fldr_outp / 'per_name_change_es.xlsx')


## Setting PCODES Dataframe


In [23]:
# 1. Setting the ADM2 PCODE information
df_pc = pd.read_excel(file_pcod, sheet_name='PCODES')
df_pc

,ADM3_ES,ADM3_PCODE,ADM2_ES,ADM2_PCODE,ADM1_ES,ADM1_PCODE
0,Abancay,PE030101,Abancay,PE0301,Apurimac,PE03
1,Abelardo Pardo Lezameta,PE020502,Bolognesi,PE0205,Ancash,PE02
2,Acari,PE040302,Caraveli,PE0403,Arequipa,PE04
3,Acas,PE021402,Ocros,PE0214,Ancash,PE02
4,Accha,PE081002,Paruro,PE0810,Cusco,PE08
...,...,...,...,...,...,...
1868,Zarumilla,PE240301,Zarumilla,PE2403,Tumbes,PE24
1869,Zepita,PE210407,Chucuito,PE2104,Puno,PE21
1870,Zorritos,PE240201,Contralmirante Villar,PE2402,Tumbes,PE24
1871,Zuñiga,PE150516,Cañete,PE1505,Lima,PE15


## Setting NAME dataframe
this is to have the right names as per the PCODE dataset

In [24]:
df_nm = pd.read_excel(file_name, sheet_name='NAMES')
df_nm

,OLD_NAME,ADM_TYPE,NEW_NAME
0,AMAZONAS,DEPA,AMAZONAS
1,BAGUA,PROV,BAGUA
2,ARAMANGO,DIST,ARAMANGO
3,COPALLIN,DIST,COPALLIN
4,EL PARCO,DIST,EL PARCO
...,...,...,...
1788,IRAZOLA,DIST,IRAZOLA
1789,NESHUYA,DIST,NESHUYA
1790,PADRE ABAD,DIST,PADRE ABAD
1791,PURUS,PROV,PURUS


## Working with the affected pop data

In [25]:
# Open excel file in a dataframe
df_pop = pd.read_excel(file_path, sheet_name='EVALUADOR', skiprows=8)

# Only workk with the wanted columns
df_pop = df_pop[['Unnamed: 0', 'DAMNIFICADA', 'AFECTADA']].copy()
col_names = ['ADM_NAME', 'DAMNIFICADA', 'AFECTADA']
df_pop.columns = col_names
df_pop = df_pop.iloc[1:] # remove the total general tab

# Remove the extra text that isn't needed
# 1. Extract DEP, PROV and DIST
df_pop['ADM_TYPE'] = df_pop['ADM_NAME'].str.extract('([A-Za-z]+)')

# 2. Remove DIST. DEPA. PROV. part from string
df_pop = df_pop.replace(regex=['DIST.'], value='')
df_pop = df_pop.replace(regex=['PROV.'], value='')
df_pop = df_pop.replace(regex=['DEPA.'], value='')

# 3. Make sure that there aren't random spaces
df_pop['ADM_NAME'] = df_pop['ADM_NAME'].str.lstrip()

# 4. Correct the name PROVIDENCIA as this was modified in the previous step
df_pop.loc[df_pop['ADM_NAME'] == 'DENCIA', 'ADM_NAME'] = 'PROVIDENCIA'
df_pop['ADM_NAME'] = np.where(df_pop['ADM_NAME'] == 'DENCIA', 'PROVIDENCIA',df_pop['ADM_NAME'] )


# 5. Create the extra columns needed
df_pop['AMD1_ES'] = ''
df_pop['AMD2_ES'] = ''
df_pop['ADM3_PCODE'] = ''

df_pop

,ADM_NAME,DAMNIFICADA,AFECTADA,ADM_TYPE,AMD1_ES,AMD2_ES,ADM3_PCODE
1,AMAZONAS,580.0,1444.0,DEPA,,,
2,BAGUA,164.0,496.0,PROV,,,
3,ARAMANGO,NaN,NaN,DIST,,,
4,COPALLIN,8.0,22.0,DIST,,,
5,EL PARCO,10.0,124.0,DIST,,,
...,...,...,...,...,...,...,...
1793,IRAZOLA,7.0,394.0,DIST,,,
1794,NESHUYA,NaN,159.0,DIST,,,
1795,PADRE ABAD,53.0,971.0,DIST,,,
1796,PURUS,NaN,NaN,PROV,,,


___
### fix step below for the whole country
___


### Fix the issue with the names

In [26]:
# merge datasets using the columns in common
df_fix = pd.merge(df_pop, df_nm, how='left', left_on=['ADM_NAME', 'ADM_TYPE'], right_on = ['OLD_NAME','ADM_TYPE'])

# replace the value in the ADM_NAME with the fix name
df_fix['ADM_NAME'] = df_fix['NEW_NAME']

df_fix

# Use the fix dataframe
# df_pop = df_fix[list(df_pop)].copy()

,ADM_NAME,DAMNIFICADA,AFECTADA,ADM_TYPE,AMD1_ES,AMD2_ES,ADM3_PCODE,OLD_NAME,NEW_NAME
0,AMAZONAS,580.0,1444.0,DEPA,,,,AMAZONAS,AMAZONAS
1,BAGUA,164.0,496.0,PROV,,,,BAGUA,BAGUA
2,ARAMANGO,NaN,NaN,DIST,,,,ARAMANGO,ARAMANGO
3,COPALLIN,8.0,22.0,DIST,,,,COPALLIN,COPALLIN
4,EL PARCO,10.0,124.0,DIST,,,,EL PARCO,EL PARCO
...,...,...,...,...,...,...,...,...,...
2152,IRAZOLA,7.0,394.0,DIST,,,,IRAZOLA,IRAZOLA
2153,NESHUYA,NaN,159.0,DIST,,,,NESHUYA,NESHUYA
2154,PADRE ABAD,53.0,971.0,DIST,,,,PADRE ABAD,PADRE ABAD
2155,PURUS,NaN,NaN,PROV,,,,PURUS,PURUS


___
___


In [27]:
# ADD DEP and PROV names to the right column
df_pop['AMD1_ES'] = np.where(df_pop['ADM_TYPE'] == 'DEPA', df_pop['ADM_NAME'],np.nan)
df_pop['AMD2_ES'] = np.where(df_pop['ADM_TYPE'] == 'PROV', df_pop['ADM_NAME'],np.nan)
df_pop

# Forward fill the DEP and PROV columns with the right names
df_pop['AMD1_ES'].fillna(method='ffill', inplace=True)
df_pop['AMD2_ES'].fillna(method='ffill', inplace=True)

# Make sure that DEPT does not have ADM2
df_pop['AMD2_ES'] = np.where(df_pop['ADM_TYPE'] == 'DEPA', '',df_pop['AMD2_ES'])

# Only work with the departmentos we are interested in
########### THIS IS THE STEP MENTIONED IN THE DESCRIPTION OF THE NOTEBOOK##############
df_pop = df_pop[df_pop.AMD1_ES.isin(['LAMBAYEQUE','PIURA','TUMBES'])]

# Remove DEPT as that isn't needed anymore
df_pop = df_pop[~df_pop['ADM_TYPE'].isin(['DEPA'])]

df_pop

,ADM_NAME,DAMNIFICADA,AFECTADA,ADM_TYPE,AMD1_ES,AMD2_ES,ADM3_PCODE
1221,CHICLAYO,2630.0,13295.0,PROV,LAMBAYEQUE,CHICLAYO,
1222,CAYALTI,6.0,213.0,DIST,LAMBAYEQUE,CHICLAYO,
1223,CHICLAYO,395.0,768.0,DIST,LAMBAYEQUE,CHICLAYO,
1224,CHONGOYAPE,166.0,1098.0,DIST,LAMBAYEQUE,CHICLAYO,
1225,ETEN,2.0,215.0,DIST,LAMBAYEQUE,CHICLAYO,
...,...,...,...,...,...,...,...
1771,ZARUMILLA,168.0,1505.0,PROV,TUMBES,ZARUMILLA,
1772,AGUAS VERDES,5.0,111.0,DIST,TUMBES,ZARUMILLA,
1773,MATAPALO,153.0,1084.0,DIST,TUMBES,ZARUMILLA,
1774,PAPAYAL,5.0,286.0,DIST,TUMBES,ZARUMILLA,


## Creating the final file for admn 3

In [28]:
df_final = df_pop.copy()

# Loop through the data to find the right value
# We are looking for match PROVINCE - DISTRICT to avoid name errors
for index, row in df_final.iterrows():
    if row['ADM_TYPE'] == 'DIST':
        # create temp dataframe
        df_temp = pd.DataFrame()
        
        # read PROV and DISTRC names
        prov_name = row['AMD2_ES']
        dist_name = row['ADM_NAME']
        
        # FOR THE FOLLOWING DISTRICT WE DON'T HAVE ADMIN BOUNDARY
        if dist_name not in ['AHUAYRO', 'PUTIS', 'UNION PROGRESO', 
                             'RIO MAGDALENA', 'NINABAMBA', 'PATIBAMBA', 
                             'KUMPIRUSHIATO', 'CIELO PUNCO', 'MANITEA', 
                             'UNION ASHANINKA', 'LAMBRAS', 'COCHABAMBA', 
                             'SAN ANTONIO', 'SANTA LUCIA', 'HUIPOCA','BOQUERON'] :
            
            
            # Set temp dataframe to be the PCODES dataframe for the PROV
            # Only one PROV at the time
            df_temp = df_pc[df_pc['ADM2_ES'].str.upper() == prov_name]
            
            # Get the PCODE for DISTRICT by reading the DIST name in temp dataframe
            print(f'Doing: {prov_name} - {dist_name}')
            idx = df_temp.index[df_temp.ADM3_ES.str.upper() == dist_name].to_list()
            pcode = df_temp.loc[idx, 'ADM3_PCODE'].values[0]
            
            print('--------------------------------')
            
            # Add PCODE to DISTRICT
            df_final.loc[index, 'ADM3_PCODE'] = pcode

# Calculate the impacted People
# Afectada + damnificada
df_final['DAMNIFICADA'] = df_final['DAMNIFICADA'].fillna(0)
df_final['AFECTADA'] = df_final['AFECTADA'].fillna(0)

df_final['IMPACTADA'] = df_final['DAMNIFICADA'] + df_final['AFECTADA']


Doing: CHICLAYO - CAYALTI
--------------------------------
Doing: CHICLAYO - CHICLAYO
--------------------------------
Doing: CHICLAYO - CHONGOYAPE
--------------------------------
Doing: CHICLAYO - ETEN
--------------------------------
Doing: CHICLAYO - ETEN PUERTO
--------------------------------
Doing: CHICLAYO - JOSE LEONARDO ORTIZ
--------------------------------
Doing: CHICLAYO - LAGUNAS
--------------------------------
Doing: CHICLAYO - LA VICTORIA
--------------------------------
Doing: CHICLAYO - MONSEFU
--------------------------------
Doing: CHICLAYO - NUEVA ARICA
--------------------------------
Doing: CHICLAYO - OYOTUN
--------------------------------
Doing: CHICLAYO - PATAPO
--------------------------------
Doing: CHICLAYO - PICSI
--------------------------------
Doing: CHICLAYO - PIMENTEL
--------------------------------
Doing: CHICLAYO - POMALCA
--------------------------------
Doing: CHICLAYO - PUCALA
--------------------------------
Doing: CHICLAYO - REQUE
-----------

## Creating the final file for admn 2


In [29]:
df_final_adm2 = df_pop.copy()

# Remove DIST as that isn't needed anymore
df_final_adm2 = df_final_adm2[~df_final_adm2['ADM_TYPE'].isin(['DIST'])]

# Add PCODE
for index, row in df_final_adm2.iterrows():
    dist_name = row['ADM_NAME']
    print(dist_name)
    idx = df_pc.index[df_pc.ADM2_ES.str.upper() == dist_name].to_list()
    pcode = df_pc.loc[idx, 'ADM2_PCODE'].values[0]
    # print(dist_name, pcode)
    df_final_adm2.loc[index, 'ADM3_PCODE'] = pcode

# Rename columns properly
col_names = ['ADM_NAME', 'DAMNIFICADA', 'AFECTADA', 'ADM_TYPE', 'AMD1_ES', 'AMD2_ES', 'ADM2_PCODE']
df_final_adm2.columns = col_names

# Calculate the impacted People
# Afectada + damnificada
df_final_adm2['DAMNIFICADA'] = df_final_adm2['DAMNIFICADA'].fillna(0)
df_final_adm2['AFECTADA'] = df_final_adm2['AFECTADA'].fillna(0)

df_final_adm2['IMPACTADA'] = df_final_adm2['DAMNIFICADA'] + df_final_adm2['AFECTADA']


df_final_adm2


CHICLAYO
FERREÑAFE
LAMBAYEQUE
AYABACA
HUANCABAMBA
MORROPON
PAITA
PIURA
SECHURA
SULLANA
TALARA
CONTRALMIRANTE VILLAR
TUMBES
ZARUMILLA


,ADM_NAME,DAMNIFICADA,AFECTADA,ADM_TYPE,AMD1_ES,AMD2_ES,ADM2_PCODE,IMPACTADA
1221,CHICLAYO,2630.0,13295.0,PROV,LAMBAYEQUE,CHICLAYO,PE1401,15925.0
1242,FERREÑAFE,1807.0,12215.0,PROV,LAMBAYEQUE,FERREÑAFE,PE1402,14022.0
1249,LAMBAYEQUE,13774.0,25009.0,PROV,LAMBAYEQUE,LAMBAYEQUE,PE1403,38783.0
1530,AYABACA,918.0,12260.0,PROV,PIURA,AYABACA,PE2002,13178.0
1541,HUANCABAMBA,3009.0,4552.0,PROV,PIURA,HUANCABAMBA,PE2003,7561.0
1550,MORROPON,1110.0,20061.0,PROV,PIURA,MORROPON,PE2004,21171.0
1561,PAITA,2178.0,12021.0,PROV,PIURA,PAITA,PE2005,14199.0
1569,PIURA,3598.0,48921.0,PROV,PIURA,PIURA,PE2001,52519.0
1580,SECHURA,504.0,8033.0,PROV,PIURA,SECHURA,PE2008,8537.0
1587,SULLANA,693.0,8149.0,PROV,PIURA,SULLANA,PE2006,8842.0


## Setting the final file

In [30]:
# Remove PROV as that isn't needed anymore for adm3
df_final = df_final[~df_final['ADM_TYPE'].isin(['PROV'])]

# Export to Excel
file_name = f'per_affd_popaffected_tab_s1_indeci_reporte_danos_lluvias_{DATA_DATE}.xlsx'

with pd.ExcelWriter(Path(fldr_outp / file_name)) as writer:  
    df_final.to_excel(writer, index=False, sheet_name='pop_impactada')
    df_final_adm2.to_excel(writer, index=False, sheet_name='pop_impactada_adm2')
    

print('--------------')
print('--------------')
print('All DONE')
print('--------------')
print('--------------')




--------------
--------------
All DONE
--------------
--------------


___
___

